# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#data science - standard library
import pandas as pd
import numpy as np

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

from api_keys import g_key
import gmaps

import requests
import json

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("NT_cities.csv")
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Lagoa,39.05,-27.98,60.89,52.72,73,1022,15.73,15.73
1,Carndonagh,55.25,-7.27,48.20,42.04,88,1014,8.77,8.77
2,Nikolskoye,59.70,30.79,45.07,35.20,81,1011,13.42,13.42
3,Ushuaia,-54.80,-68.30,55.40,42.04,47,994,18.34,18.34
4,Xifeng,42.74,124.72,34.29,21.25,69,1026,15.23,15.23


In [4]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humids = df["Humidity"].astype(float)
locations 

,Latitude,Longitude
0,39.05,-27.98
1,55.25,-7.27
2,59.70,30.79
3,-54.80,-68.30
4,42.74,124.72
...,...,...
542,26.54,-77.06
543,5.73,102.49
544,-6.72,36.37
545,15.32,78.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
coords = df[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coords, info_box_content=states)
# # Add the layer to the map
# fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
df.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Lagoa,39.05,-27.98,60.89,52.72,73,1022,15.73,15.73
1,Carndonagh,55.25,-7.27,48.20,42.04,88,1014,8.77,8.77
2,Nikolskoye,59.70,30.79,45.07,35.20,81,1011,13.42,13.42
3,Ushuaia,-54.80,-68.30,55.40,42.04,47,994,18.34,18.34
4,Xifeng,42.74,124.72,34.29,21.25,69,1026,15.23,15.23


In [7]:
df_ideal = df[(df['Temperature'] >= 70) & (df['Temperature'] <= 78)]
df_ideal = df_ideal[df_ideal['Wind Speed'] < 5]
df_ideal = df_ideal[df_ideal['Humidity'] < 70]
hotel = (df_ideal.reset_index(drop=True))
hotel

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Lakhīmpur,27.95,80.77,73.65,72.54,45,1019,2.59,2.59
1,Kabin Buri,13.95,101.72,77.99,81.41,69,1015,4.90,4.90
2,Hong Kong,22.29,114.16,77.07,79.84,56,1018,1.01,1.01
3,Panzhihua,26.59,101.71,74.61,75.13,49,1024,1.36,1.36
4,Mushābani,22.52,86.45,73.40,72.37,53,1019,4.70,4.70
5,Khanbari,27.37,87.20,75.74,72.81,37,1021,4.25,4.25
6,Wau,7.70,28.00,77.09,80.13,57,1012,0.87,0.87
7,Enshi,30.30,109.48,71.74,70.84,44,1027,1.12,1.12
8,Kibakwe,-6.72,36.37,73.40,74.55,63,1017,3.78,3.78


In [8]:
len(df_ideal)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel['Hotel Name'] = ""
hotel

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness,Hotel Name
0,Lakhīmpur,27.95,80.77,73.65,72.54,45,1019,2.59,2.59,
1,Kabin Buri,13.95,101.72,77.99,81.41,69,1015,4.90,4.90,
2,Hong Kong,22.29,114.16,77.07,79.84,56,1018,1.01,1.01,
3,Panzhihua,26.59,101.71,74.61,75.13,49,1024,1.36,1.36,
4,Mushābani,22.52,86.45,73.40,72.37,53,1019,4.70,4.70,
5,Khanbari,27.37,87.20,75.74,72.81,37,1021,4.25,4.25,
6,Wau,7.70,28.00,77.09,80.13,57,1012,0.87,0.87,
7,Enshi,30.30,109.48,71.74,70.84,44,1027,1.12,1.12,
8,Kibakwe,-6.72,36.37,73.40,74.55,63,1017,3.78,3.78,


In [10]:
for index, row in hotel.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel.loc[index, "Hotel Name"] = "NaN"
        
hotel

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness,Hotel Name
0,Lakhīmpur,27.95,80.77,73.65,72.54,45,1019,2.59,2.59,Hotel RR Grand
1,Kabin Buri,13.95,101.72,77.99,81.41,69,1015,4.90,4.90,Kantary Hotel Kabinburi
2,Hong Kong,22.29,114.16,77.07,79.84,56,1018,1.01,1.01,K11 ARTUS
3,Panzhihua,26.59,101.71,74.61,75.13,49,1024,1.36,1.36,Jinsha Pearl Hotel
4,Mushābani,22.52,86.45,73.40,72.37,53,1019,4.70,4.70,NaN
5,Khanbari,27.37,87.20,75.74,72.81,37,1021,4.25,4.25,Hotel Barun Sankhuwasabha
6,Wau,7.70,28.00,77.09,80.13,57,1012,0.87,0.87,Amarula Lodge and Restaurant
7,Enshi,30.30,109.48,71.74,70.84,44,1027,1.12,1.12,Moevenpick Enshi
8,Kibakwe,-6.72,36.37,73.40,74.55,63,1017,3.78,3.78,NaN


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…